To be able to use scikit-learn with xgb, we need to downgrade scikit-learn

After this is done, please restart your runtime via Runtime -> Restart

In [ ]:
# Install compatible scikit-learn since xgboost has not updated their library yet
!pip uninstall -y scikit-learn
!pip install "scikit-learn<1.6"

  Using cached scikit_learn-1.5.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
Using cached scikit_learn-1.5.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.3 MB)


# Imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
import joblib

# Download Dataset

In [ ]:
!if [ ! -d "powertheft" ]; then \
    sudo apt-get update && \
    sudo apt-get install -y aria2 zip unzip  && \
    aria2c -x 8 -s 8 -d . -o powertheft.zip "https://www.kaggle.com/api/v1/datasets/download/ahmedrady66/theft-detection-scheme-in-smart-grids" && \
    unzip powertheft.zip -d "powertheft"; \
fi

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,199 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,640 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jamm

# Load Data and Extract Features

In [ ]:
data = pd.read_csv('powertheft/AllData/AllData.csv')
display(data)

,UserId,IsStealer,1/1/2014,1/2/2014,1/3/2014,1/4/2014,1/5/2014,1/6/2014,1/7/2014,1/8/2014,...,10/22/2016,10/23/2016,10/24/2016,10/25/2016,10/26/2016,10/27/2016,10/28/2016,10/29/2016,10/30/2016,10/31/2016
0,00002188D4496609AC58502A1241C0E0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,3.84,3.93,3.42,3.27,3.78,3.30,3.54,2.12,2.81,2.78
1,0000E78A22CB04533A0D9E1F2FBEEC5D,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0002D8E9C198E4A2B03BFA6D1E2E1B6D,0,8.04,5.94,11.96,5.09,4.72,7.21,8.47,8.06,...,9.87,7.46,7.36,7.70,7.33,7.87,7.98,7.52,6.78,10.55
3,0002FAFFBC621C87127A59E907D16010,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,000395F84A94D4CB2E5D4D77372CFB4D,1,22.23,22.46,20.13,21.79,24.00,24.00,19.10,15.91,...,24.20,18.89,3.96,19.54,19.32,37.87,26.44,9.49,14.28,22.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44213,FFFBCB1DC562EB890DDCC74B3BE7C1AE,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
44214,FFFD453B82863B29301AE31059863359,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.11,0.00,0.13,0.10,0.17,0.64,0.15
44215,FFFDCC5003A692984315154FA7409B60,0,2.95,0.00,0.00,10.73,0.00,0.00,4.43,3.63,...,20.47,16.03,13.12,11.66,12.61,12.81,14.87,17.24,13.03,11.83
44216,FFFDE28BE9E1DC2A8ED599282AF34B33,0,4.09,10.43,0.00,10.51,7.38,2.68,0.00,1.72,...,7.12,5.43,4.79,4.26,4.32,1.08,5.88,5.70,4.11,4.64


In [ ]:
features = []
labels = []

# Extract consumption columns (starting from the 3rd column to the last one)
consumption_columns = data.iloc[:, 2:]

for i in range(consumption_columns.shape[0]):  # Iterate over each user
    user = consumption_columns.iloc[i, :].to_numpy()
    mean_consumption = user.mean()
    max_consumption = user.max()
    min_consumption = user.min()
    std_consumption = user.std()
    sum_consumption = user.sum()

    feature_vector = np.array([mean_consumption, max_consumption, min_consumption, std_consumption, sum_consumption])

    features.append(feature_vector)
    labels.append(data['IsStealer'].iloc[i])


X = np.array(features)
y = np.array(labels)

print(f"Feature matrix shape: {X.shape}")
print(f"Labels shape: {y.shape}")

Feature matrix shape: (44218, 5)
Labels shape: (44218,)


# Split Data

In [ ]:
TRAIN_TEST_SPLIT = 0.9

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=1 - TRAIN_TEST_SPLIT, random_state=42)

print(f"Training set shape: {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")

Training set shape: (39796, 5)
Validation set shape: (4422, 5)


# Design and Train Model

In [ ]:
model = xgb.XGBClassifier(
    n_estimators=100,         # Number of boosting rounds
    learning_rate=0.1,        # Learning rate
    max_depth=6,              # Depth of trees
    objective='binary:logistic',  # Binary classification
    eval_metric='logloss'     # Evaluation metric (logarithmic loss)
)

model.fit(X_train, y_train)

# Evaluate the model
print("Train Accuracy:", accuracy_score(y_train, model.predict(X_train)))
print("Validation Accuracy:", accuracy_score(y_val, model.predict(X_val)))
print("Classification Report:\n", classification_report(y_val, model.predict(X_val)))

Train Accuracy: 0.9161724796461956
Validation Accuracy: 0.9151967435549525
Classification Report:
               precision    recall  f1-score   support

           0       0.92      1.00      0.96      4046
           1       0.52      0.03      0.06       376

    accuracy                           0.92      4422
   macro avg       0.72      0.51      0.51      4422
weighted avg       0.88      0.92      0.88      4422



In [ ]:
# Similar to a training loop, let's use grid search to find the best parameters and optimize over them
param_grid = {
    'max_depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [50, 100, 200],
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1)
grid_search.fit(X_train, y_train)

# Print best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best Parameters: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100}
Best Score: 0.9149160668083206


The best score is actually the parameters we initially chose, so there is no need to modify anything.

In [ ]:
# Save model to perform inference
joblib.dump(model, 'power_theft_model.pkl')

['power_theft_model.pkl']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import locale
locale.getpreferredencoding = lambda: "UTF-8"
! cp "/content/power_theft_model.pkl" /content/drive/MyDrive/

Mounted at /content/drive
